# SGR Full Debug: Последний Run

Этот ноутбук делает полный разбор **последнего scan-run** из `dialogs.db`:
- что получилось,
- что не получилось,
- где есть пропуски/ошибки,
- какие кейсы judge посчитал плохими.

In [6]:
from pathlib import Path
import json
import sqlite3
import textwrap

import pandas as pd

pd.set_option("display.max_colwidth", 220)
pd.set_option("display.max_rows", 200)

DB_PATH = '../dialogs.db'

conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row

def qdf(sql: str, params: tuple = ()) -> pd.DataFrame:
    return pd.read_sql_query(sql, conn, params=params)

print(f"DB: {DB_PATH}")


DB: ../dialogs.db


## 1) Последний run: snapshot

In [7]:
run_df = qdf(
    """
    SELECT run_id, status, model, conversation_from, conversation_to,
           selected_conversations, messages_count, started_at_utc, finished_at_utc, summary_json
    FROM scan_runs
    ORDER BY started_at_utc DESC
    LIMIT 1
    """
)

HAS_RUN = not run_df.empty
if not HAS_RUN:
    print("Нет scan-run в БД. Запустите: make scan")
else:
    run_row = run_df.iloc[0].to_dict()
    RUN_ID = str(run_row["run_id"])
    summary = json.loads(run_row.get("summary_json") or "{}")

    started = pd.to_datetime(run_row["started_at_utc"], utc=True, errors="coerce")
    finished = pd.to_datetime(run_row["finished_at_utc"], utc=True, errors="coerce")
    duration_sec = None
    if pd.notna(started) and pd.notna(finished):
        duration_sec = (finished - started).total_seconds()

    snapshot = {
        "run_id": RUN_ID,
        "status": run_row.get("status"),
        "model": run_row.get("model"),
        "conversation_range": f"{run_row.get('conversation_from')}..{run_row.get('conversation_to')}",
        "selected_conversations": run_row.get("selected_conversations"),
        "messages_total": run_row.get("messages_count"),
        "seller_messages": summary.get("seller_messages"),
        "customer_context_only": summary.get("customer_messages_context_only"),
        "processed": summary.get("processed"),
        "inserted": summary.get("inserted"),
        "judged": summary.get("judged"),
        "skipped_due_to_errors": summary.get("skipped_due_to_errors"),
        "evidence_mismatch_skipped": summary.get("evidence_mismatch_skipped"),
        "schema_errors": summary.get("schema_errors"),
        "non_schema_errors": summary.get("non_schema_errors"),
        "duration_sec": duration_sec,
        "started_at_utc": run_row.get("started_at_utc"),
        "finished_at_utc": run_row.get("finished_at_utc"),
    }

    pd.DataFrame([snapshot])

## 2) LLM pipeline health (calls/errors/coverage)

In [8]:
if not HAS_RUN:
    print("Нет данных")
else:
    llm_health = qdf(
        """
        SELECT phase, rule_key, attempt,
               COUNT(*) AS calls,
               SUM(CASE WHEN error_message<>'' THEN 1 ELSE 0 END) AS errors,
               SUM(CASE WHEN parse_ok=0 THEN 1 ELSE 0 END) AS parse_fail,
               SUM(CASE WHEN validation_ok=0 THEN 1 ELSE 0 END) AS validation_fail,
               ROUND(AVG(latency_ms), 1) AS avg_latency_ms
        FROM llm_calls
        WHERE run_id=?
        GROUP BY phase, rule_key, attempt
        ORDER BY phase, rule_key, attempt
        """,
        (RUN_ID,),
    )

    seller_messages = int(summary.get("seller_messages") or 0)
    expected_eval_calls = seller_messages * 3
    actual_eval_calls = int(
        qdf("SELECT COUNT(*) AS c FROM llm_calls WHERE run_id=? AND phase='evaluator'", (RUN_ID,)).iloc[0]["c"]
    )
    inserted = int(summary.get("inserted") or 0)
    judged = int(summary.get("judged") or 0)
    dropped_cases = max(actual_eval_calls - inserted, 0)
    judge_coverage = (judged / inserted) if inserted else 0.0

    coverage = pd.DataFrame([
        {
            "expected_evaluator_calls": expected_eval_calls,
            "actual_evaluator_calls": actual_eval_calls,
            "inserted_results": inserted,
            "dropped_cases": dropped_cases,
            "judged_results": judged,
            "judge_coverage": round(judge_coverage, 4),
        }
    ])

    print("### Aggregated health")
    display(coverage)
    print("\n### LLM calls breakdown")
    display(llm_health)


### Aggregated health


,expected_evaluator_calls,actual_evaluator_calls,inserted_results,dropped_cases,judged_results,judge_coverage
0,147,148,146,2,146,1.0



### LLM calls breakdown


,phase,rule_key,attempt,calls,errors,parse_fail,validation_fail,avg_latency_ms
0,evaluator,empathy,1,49,0,0,0,2200.4
1,evaluator,greeting,1,49,0,0,0,1724.4
2,evaluator,upsell,1,49,0,0,0,1841.4
3,evaluator,upsell,2,1,0,0,0,2109.0
4,judge,empathy,1,49,0,0,0,1975.9
5,judge,greeting,1,49,0,0,0,1632.1
6,judge,upsell,1,48,0,0,0,1735.2


## 3) Качество по правилам (таблично)

In [9]:
if not HAS_RUN:
    print("Нет данных")
else:
    metrics_long = qdf(
        """
        SELECT rule_key, metric_name, metric_value
        FROM scan_metrics
        WHERE run_id=?
        """,
        (RUN_ID,),
    )

    if metrics_long.empty:
        print("Для run нет метрик")
    else:
        metrics = metrics_long.pivot_table(
            index="rule_key",
            columns="metric_name",
            values="metric_value",
            aggfunc="first",
        ).reset_index()

        cols_order = [
            "rule_key", "accuracy", "precision", "recall", "f1", "coverage",
            "tp", "fp", "tn", "fn", "total"
        ]
        for c in cols_order:
            if c not in metrics.columns:
                metrics[c] = pd.NA
        metrics = metrics[cols_order].sort_values(by=["f1", "accuracy"], ascending=False)

        styled = metrics.style.format(
            {
                "accuracy": "{:.3f}",
                "precision": "{:.3f}",
                "recall": "{:.3f}",
                "f1": "{:.3f}",
                "coverage": "{:.3f}",
            }
        ).background_gradient(subset=["accuracy", "precision", "recall", "f1", "coverage"], cmap="RdYlGn")

        display(styled)

ImportError: background_gradient requires matplotlib.

## 4) Judge bad-cases (judge_label = 0)

In [10]:
TOP_N = 25

if not HAS_RUN:
    print("Нет данных")
else:
    bad_cases = qdf(
        """
        SELECT r.rule_key, r.conversation_id, r.message_id,
               m.speaker_label, m.text,
               r.eval_hit, r.eval_confidence, r.judge_confidence,
               r.eval_reason, r.judge_rationale
        FROM scan_results r
        JOIN messages m ON m.message_id = r.message_id
        WHERE r.run_id=?
          AND r.judge_label=0
        ORDER BY r.rule_key, r.message_id
        """,
        (RUN_ID,),
    )

    print(f"Всего bad-cases judge_label=0: {len(bad_cases)}")
    display(bad_cases.head(TOP_N))

    print("\n### bad-cases, где evaluator сработал (eval_hit=1):")
    display(bad_cases[bad_cases["eval_hit"] == 1].head(TOP_N))

    print("\n### bad-cases, где evaluator не сработал (eval_hit=0):")
    display(bad_cases[bad_cases["eval_hit"] == 0].head(TOP_N))


Всего bad-cases judge_label=0: 11


,rule_key,conversation_id,message_id,speaker_label,text,eval_hit,eval_confidence,judge_confidence,eval_reason,judge_rationale
0,empathy,modamart__0_transcript,4946,Sales Rep,"Hi there! Thank you for taking the time to speak with me today. My name is Jamie, and I’m a sales representative from ModaMart. How are you today?",1,0.95,0.90,"Продавец явно выражает эмпатию, задавая вопрос о самочувствии клиента, что соответствует правилу и уместно в начале разговора.","Evaluator неправильно применил правило, так как в сообщении продавец просто здоровается и задает стандартный приветственный вопрос, что является обычной вежливостью, но не выражает явно эмпатию. В начале диалога подо..."
1,empathy,modamart__0_transcript,4948,Sales Rep,"I’m great, thank you! So, I understand you’ve been browsing our online store. Is there anything specific you’re looking for today?",1,0.90,0.90,"В реплике продавца выражена эмпатия через фразу ""I’m great, thank you!"", которая отвечает на вопрос клиента, а далее корректно проявляется заинтересованность и приглашение к диалогу, что соответствует правилу empathy...","В реплике продавца выражена вежливость и проявлена заинтересованность, но непосредственно эмпатия, то есть эмоциональное понимание или сочувствие к состоянию клиента, не выражена явно. Фраза ""I’m great, thank you!"" с..."
2,empathy,modamart__1_transcript,4981,Sales Rep,** Great choice! I’ll get that set up for you with express shipping. You should receive it within 2-3 business days. Can I assist you with anything else today?,1,0.90,0.90,"В текущем сообщении продавец выражает поддержку и одобрение выбора клиента («Great choice!»), сообщает полезную информацию о сроках доставки и вежливо предлагает дополнительную помощь, что соответствует проявлению ум...","В сообщении продавца отсутствует явное выражение эмпатии в контексте переживаний или эмоциональных потребностей клиента. Выражение «Great choice!» больше относится к поддержке и одобрению, но не отражает глубокого по..."
3,empathy,modamart__1_transcript,4983,Sales Rep,"** You're welcome, John! If you have any other questions, feel free to contact us. Have a great day!",1,0.90,0.90,"Продавец вежливо и персонально обращается к клиенту по имени, выражает готовность помочь и желает хорошего дня, что соответствует проявлению эмпатии в данном завершительном сообщении.","Хотя сообщение вежливое и персональное, оно носит стандартный, формальный характер прощания, не выражая глубокой эмпатии или понимания чувств клиента на основе контекста диалога. Эмпатия подразумевает явное проявлени..."
4,empathy,modamart__2_transcript,4985,Sales Rep,"Hi there, this is Jake from ModaMart. How are you today?",1,0.90,0.95,"В реплике продавца явно выражена эмпатия через приветствие и вопрос о самочувствии клиента, что уместно в начале диалога.","В приведённом контексте текущим сообщением является первая реплика продавца, которая одновременно является и приветствием, и вопросом. Но правило требует, чтобы эмпатия была уместна и явно выражена с учётом предшеств..."
5,empathy,modamart__2_transcript,4999,Sales Rep,I'm glad you asked! We currently have a 15% off promotion for first-time online customers. I can send you a discount code if you're interested.,1,0.90,0.95,"Продавец явно выражает положительную реакцию на вопрос клиента, что является проявлением эмпатии и поддерживает дружелюбный и конструктивный диалог.","В текущем сообщении продавца выражена радость по поводу вопроса клиента и предложение скидки, что является информативным и дружелюбным, но не выражает эмпатию по контексту диалога. Эмпатия должна отображать понимание..."
6,empathy,modamart__2_transcript,5003,Sales Rep,"You're welcome! Don't hesitate to reach out if you have any more questions, and enjoy shopping at ModaMart. Have a great day!",1,0.90,0.90,"The message clearly conveys empathy by warmly thanking the customer and encouraging future contact, which is appropriate and supportive at the end of the conversation.","The evaluator marked the empathy crit


### bad-cases, где evaluator сработал (eval_hit=1):


,rule_key,conversation_id,message_id,speaker_label,text,eval_hit,eval_confidence,judge_confidence,eval_reason,judge_rationale
0,empathy,modamart__0_transcript,4946,Sales Rep,"Hi there! Thank you for taking the time to speak with me today. My name is Jamie, and I’m a sales representative from ModaMart. How are you today?",1,0.95,0.90,"Продавец явно выражает эмпатию, задавая вопрос о самочувствии клиента, что соответствует правилу и уместно в начале разговора.","Evaluator неправильно применил правило, так как в сообщении продавец просто здоровается и задает стандартный приветственный вопрос, что является обычной вежливостью, но не выражает явно эмпатию. В начале диалога подо..."
1,empathy,modamart__0_transcript,4948,Sales Rep,"I’m great, thank you! So, I understand you’ve been browsing our online store. Is there anything specific you’re looking for today?",1,0.90,0.90,"В реплике продавца выражена эмпатия через фразу ""I’m great, thank you!"", которая отвечает на вопрос клиента, а далее корректно проявляется заинтересованность и приглашение к диалогу, что соответствует правилу empathy...","В реплике продавца выражена вежливость и проявлена заинтересованность, но непосредственно эмпатия, то есть эмоциональное понимание или сочувствие к состоянию клиента, не выражена явно. Фраза ""I’m great, thank you!"" с..."
2,empathy,modamart__1_transcript,4981,Sales Rep,** Great choice! I’ll get that set up for you with express shipping. You should receive it within 2-3 business days. Can I assist you with anything else today?,1,0.90,0.90,"В текущем сообщении продавец выражает поддержку и одобрение выбора клиента («Great choice!»), сообщает полезную информацию о сроках доставки и вежливо предлагает дополнительную помощь, что соответствует проявлению ум...","В сообщении продавца отсутствует явное выражение эмпатии в контексте переживаний или эмоциональных потребностей клиента. Выражение «Great choice!» больше относится к поддержке и одобрению, но не отражает глубокого по..."
3,empathy,modamart__1_transcript,4983,Sales Rep,"** You're welcome, John! If you have any other questions, feel free to contact us. Have a great day!",1,0.90,0.90,"Продавец вежливо и персонально обращается к клиенту по имени, выражает готовность помочь и желает хорошего дня, что соответствует проявлению эмпатии в данном завершительном сообщении.","Хотя сообщение вежливое и персональное, оно носит стандартный, формальный характер прощания, не выражая глубокой эмпатии или понимания чувств клиента на основе контекста диалога. Эмпатия подразумевает явное проявлени..."
4,empathy,modamart__2_transcript,4985,Sales Rep,"Hi there, this is Jake from ModaMart. How are you today?",1,0.90,0.95,"В реплике продавца явно выражена эмпатия через приветствие и вопрос о самочувствии клиента, что уместно в начале диалога.","В приведённом контексте текущим сообщением является первая реплика продавца, которая одновременно является и приветствием, и вопросом. Но правило требует, чтобы эмпатия была уместна и явно выражена с учётом предшеств..."
5,empathy,modamart__2_transcript,4999,Sales Rep,I'm glad you asked! We currently have a 15% off promotion for first-time online customers. I can send you a discount code if you're interested.,1,0.90,0.95,"Продавец явно выражает положительную реакцию на вопрос клиента, что является проявлением эмпатии и поддерживает дружелюбный и конструктивный диалог.","В текущем сообщении продавца выражена радость по поводу вопроса клиента и предложение скидки, что является информативным и дружелюбным, но не выражает эмпатию по контексту диалога. Эмпатия должна отображать понимание..."
6,empathy,modamart__2_transcript,5003,Sales Rep,"You're welcome! Don't hesitate to reach out if you have any more questions, and enjoy shopping at ModaMart. Have a great day!",1,0.90,0.90,"The message clearly conveys empathy by warmly thanking the customer and encouraging future contact, which is appropriate and supportive at the end of the conversation.","The evaluator marked the empathy crit


### bad-cases, где evaluator не сработал (eval_hit=0):


,rule_key,conversation_id,message_id,speaker_label,text,eval_hit,eval_confidence,judge_confidence,eval_reason,judge_rationale
10,upsell,modamart__2_transcript,4999,Sales Rep,I'm glad you asked! We currently have a 15% off promotion for first-time online customers. I can send you a discount code if you're interested.,0,0.9,0.95,"Сообщение предлагает скидку, но не содержит предложения дополнительной опции, тарифа или пакета, уместного контексту, необходимого для upsell.","Сообщение Sales Rep предлагает скидку 15% для новых онлайн клиентов, что является промоакцией, но не предлагает дополнительной опции, тарифа или пакета, то есть не соответствует критериям upsell. Однако, evaluator за..."


## 5) Disagreements: evaluator vs judge

In [11]:
if not HAS_RUN:
    print("Нет данных")
else:
    disagree = qdf(
        """
        SELECT r.rule_key, r.conversation_id, r.message_id,
               m.speaker_label, m.text,
               r.eval_hit, r.judge_label,
               r.eval_confidence, r.judge_confidence,
               r.eval_reason, r.judge_rationale
        FROM scan_results r
        JOIN messages m ON m.message_id = r.message_id
        WHERE r.run_id=?
          AND r.judge_label IS NOT NULL
          AND r.eval_hit <> r.judge_label
        ORDER BY r.rule_key, r.message_id
        """,
        (RUN_ID,),
    )

    if disagree.empty:
        print("Расхождений evaluator vs judge нет")
    else:
        disagree["confidence_gap"] = (disagree["eval_confidence"] - disagree["judge_confidence"]).abs()
        disagree = disagree.sort_values("confidence_gap", ascending=False)
        print(f"Всего disagreements: {len(disagree)}")
        display(disagree.head(TOP_N))

Всего disagreements: 105


,rule_key,conversation_id,message_id,speaker_label,text,eval_hit,judge_label,eval_confidence,judge_confidence,eval_reason,judge_rationale,confidence_gap
18,empathy,modamart__4_transcript,5032,Sales Rep,"Yes, we do offer financing options. You can break down the cost into manageable monthly payments at 0% interest if paid within 6 months. We also have a 30-day return policy if you’re not completely satisfied with you...",0,1,0.70,0.95,В сообщении продавца нет выраженной эмпатии — отсутствует признание опасений клиента или эмоциональный отклик на его сомнения. Текст содержит только информацию о способах оплаты и возврате без выражения понимания или...,"Evaluator правильно отметил отсутствие эмпатии в сообщении продавца. Ответ ограничивается информированием о вариантах оплаты и политике возврата без выражения понимания или эмоциональной поддержки клиенту, что не соо...",0.25
76,upsell,modamart__1_transcript,4967,Sales Rep,"** Hi John! Thanks for reaching out. I’d be happy to help you find the perfect jacket. Are you looking for something specific, like a certain material or style?",0,1,0.90,1.00,"В сообщении отсутствует предложение дополнительной опции, тарифа или пакета; нет уместного к контексту допродажи.","В сообщении продавец предлагает помочь подобрать куртку, задавая уточняющий вопрос, но не предлагает дополнительную опцию, тариф или пакет, что соответствует правилу для допродажи. Решение evaluator верно.",0.10
4,empathy,modamart__1_transcript,4979,Sales Rep,"** In that case, I’d recommend our express shipping option. Let’s finalize your choice of jacket first. Are you leaning towards a specific model?",0,1,0.80,0.90,"В данном сообщении продавец предоставляет рекомендацию и задает уточняющий вопрос, но не выражает явной эмпатии, поскольку не отражает чувства или состояние клиента в ответе.",Evaluator correctly identified that the sales rep's message provides recommendation and asks a clarifying question but does not explicitly express empathy by acknowledging the customer's feelings or concerns in this ...,0.10
96,upsell,modamart__4_transcript,5022,Sales Rep,"Good afternoon, thank you for calling ModaMart. My name is Sarah, how can I assist you today?",0,1,0.90,1.00,"В данном сообщении отсутствует предложение дополнительной опции, тарифа или пакета, уместное к контексту. Это стандартное приветствие и предложение помощи, без попытки допродажи.","В сообщении отсутствует предложение дополнительной опции, тарифа или пакета. Это стандартное приветствие и предложение помощи, поэтому ответ evaluator корректен.",0.10
14,empathy,modamart__3_transcript,5017,Sales Rep,"Great to hear! You can visit our website and sign up for the ModaMart Rewards program right away. If you need any assistance with sizing or product details, feel free to reach out to our customer service team. They'l...",1,0,0.90,0.80,"The sales rep shows positive empathy by responding warmly to the customer's willingness to try, encouraging further contact for assistance, thus reducing friction and supporting the customer's decision.","The evaluator marked the message as empathetic, but the Sales Rep's message at [13] is mostly informational and promotional without expressing clear empathy. It does not explicitly acknowledge or respond to the custo...",0.10
25,greeting,modamart__0_transcript,4952,Sales Rep,"That makes total sense. Nobody likes to feel weighed down. We have a range of jackets filled with high-loft down, which is incredibly warm yet very lightweight. Have you had a chance to read any reviews or check out ...",0,1,1.00,0.95,В сообщении нет явного приветствия клиенту; первое приветствие было в сообщении 1.,"В указанном сообщении продавца отсутствует явное приветствие клиенту. Приветствие уже было сделано в предыдущем сообщении (1), а текущее сообщение содержит продолжение диалога без повторного приветствия, что соответс...",0.05
38,greeting,modamart__1_transcript,4981,Sales Rep,** Great choice! I’ll get that set up for you with express shipping. You shoul

## 6) Dropped/Skipped кейсы (вызов evaluator был, результата нет)

In [12]:
if not HAS_RUN:
    print("Нет данных")
else:
    dropped = qdf(
        """
        WITH e AS (
          SELECT run_id, conversation_id, message_id, rule_key, MIN(call_id) AS call_id
          FROM llm_calls
          WHERE run_id=? AND phase='evaluator'
          GROUP BY run_id, conversation_id, message_id, rule_key
        )
        SELECT e.rule_key, e.conversation_id, e.message_id, m.speaker_label, m.text, e.call_id
        FROM e
        LEFT JOIN scan_results r
          ON r.run_id=e.run_id AND r.message_id=e.message_id AND r.rule_key=e.rule_key
        JOIN messages m ON m.message_id = e.message_id
        WHERE r.result_id IS NULL
        ORDER BY e.call_id
        """,
        (RUN_ID,),
    )

    print(f"Dropped/Skipped кейсов: {len(dropped)}")
    display(dropped.head(100))

Dropped/Skipped кейсов: 1


,rule_key,conversation_id,message_id,speaker_label,text,call_id
0,upsell,modamart__3_transcript,5017,Sales Rep,"Great to hear! You can visit our website and sign up for the ModaMart Rewards program right away. If you need any assistance with sizing or product details, feel free to reach out to our customer service team. They'l...",110


## 7) Авто-вывод: что получилось / что не получилось

In [13]:
if not HAS_RUN:
    print("Нет run для интерпретации")
else:
    schema_errors = int(summary.get("schema_errors") or 0)
    evidence_skips = int(summary.get("evidence_mismatch_skipped") or 0)
    skipped = int(summary.get("skipped_due_to_errors") or 0)
    inserted = int(summary.get("inserted") or 0)
    judged = int(summary.get("judged") or 0)
    judge_coverage = (judged / inserted) if inserted else 0.0

    metrics_short = qdf(
        """
        SELECT rule_key,
               MAX(CASE WHEN metric_name='precision' THEN metric_value END) AS precision,
               MAX(CASE WHEN metric_name='recall' THEN metric_value END) AS recall,
               MAX(CASE WHEN metric_name='f1' THEN metric_value END) AS f1
        FROM scan_metrics
        WHERE run_id=?
        GROUP BY rule_key
        ORDER BY f1 DESC
        """,
        (RUN_ID,),
    )

    good = []
    bad = []
    actions = []

    if schema_errors == 0:
        good.append("Schema-level ошибок нет: JSON schema/validation контур устойчив.")
    else:
        bad.append(f"Есть schema-level ошибки: {schema_errors}.")

    if judge_coverage >= 0.95:
        good.append(f"Judge coverage высокий: {judge_coverage:.1%}.")
    else:
        bad.append(f"Judge coverage низкий: {judge_coverage:.1%}.")

    if not metrics_short.empty:
        top_rule = metrics_short.iloc[0]
        good.append(f"Лучшее правило по F1: {top_rule['rule_key']} (F1={top_rule['f1']:.3f}).")

        weak = metrics_short[(metrics_short["precision"] < 0.60) | (metrics_short["recall"] < 0.60)]
        if not weak.empty:
            bad.append("Есть правила с низким precision/recall: " + ", ".join(weak["rule_key"].tolist()))

    if evidence_skips > 0:
        bad.append(f"Пропуски из-за evidence mismatch после retry: {evidence_skips}.")
        actions.append("Проверить dropped/skip кейсы и усилить prompt-инструкцию для evidence по проблемным rule_key.")

    if skipped > 0:
        bad.append(f"Общий счетчик skipped_due_to_errors: {skipped}.")
        actions.append("Сверить llm_calls (attempt=2) и тексты сообщений, где отсутствует запись в scan_results.")

    actions.append("Просмотреть топ-25 bad-cases judge_label=0 и выделить 3 повторяющихся паттерна ошибок.")
    actions.append("Для правил с низким recall подготовить примеры missed-cases и скорректировать wording в SGR prompt.")

    print("Что получилось:")
    for x in good:
        print("-", x)

    print("\nЧто не получилось:")
    if bad:
        for x in bad:
            print("-", x)
    else:
        print("- Критичных проблем по текущему run не найдено.")

    print("\nЧто проверить дальше:")
    for x in actions[:3]:
        print("-", x)


Что получилось:
- Schema-level ошибок нет: JSON schema/validation контур устойчив.
- Judge coverage высокий: 100.0%.
- Лучшее правило по F1: empathy (F1=0.693).

Что не получилось:
- Есть правила с низким precision/recall: upsell, greeting
- Пропуски из-за evidence mismatch после retry: 1.
- Общий счетчик skipped_due_to_errors: 1.

Что проверить дальше:
- Проверить dropped/skip кейсы и усилить prompt-инструкцию для evidence по проблемным rule_key.
- Сверить llm_calls (attempt=2) и тексты сообщений, где отсутствует запись в scan_results.
- Просмотреть топ-25 bad-cases judge_label=0 и выделить 3 повторяющихся паттерна ошибок.


In [ ]:
conn.close()